# RFM Analysis on Bank Customers

The project aims to examine the demographic characteristics of the bank's customers, perform segmentation based on their deposits in the bank, and provide possible recommendations and tactics to utilize the services the bank offers to its depositors in the most optimal way possible.

Nowadays, traditional banks face challenges from emerging digital banks and competitors who employ targeted strategies to personalize their products and services in order to retain customers and depositors. Furthermore, due to limited resources, it is crucial for banks to identify the most valuable customer groups, loyal customers, and high-potential groups for future development.

The RFM approach which stands for (Recency, Frequency, and Monetary), can serve as a valuable and intuitive approach for banks to categorize their customers and address their business questions and problems. In this analysis, the analyst will utilize sample data to create customer segments based on their transactional behavior and provide recommendations on how bank managers can develop targeted plans for each group to maximize profitability.t.

The content in the project:
1. **RFM Model:** Recency, Frequency, Monetary Score. This is a customer segmentation technique used by businesses to understand and classify their customer base based on their transactional behavior. It is a data-driven approach that provides insights into customer value, engagement, and potential.
2. **Pareto Analysis:** How  many customers contribute to the most transaction volume? In Pareto analysis, data is collected and analyzed to determine which factors or inputs have the most significant impact on a particular outcome. This analysis helps in identifying the "vital few" factors that contribute the most to the desired outcome, as well as the "trivial many" factors that have relatively little impact.
3.  **Customer Strategy:**- This is  based on the segmentation and analysis result.,The possibleh strategiesmightd the bank utilize to better allocate its servicing recourses and customizeitsr offerings to the customer..

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set the display format to two decimal places for all floating-point numbers when printing or displaying a DataFrame in pandas.
pd.set_option('display.float_format',lambda x: '%.2f' % x)

# To hide warning messages that you consider to be non-critical or distracting.
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Data Cleaning

Import the dataset using the `pd.read_csv()` function.

In [ ]:
# Import the data
data = pd.read_csv("/kaggle/input/bank-customer-segmentation/bank_transactions.csv")

data.head()

In [ ]:
print('There are ', len(data), ' transactions/rows and ', data.shape[1], ' columns in our dataset.')

data.count()

We see that there are 1,048,567 transactions (rows) but some columns have lower counts. This indicates that there are null (empty) values in those columns.

Check for the missing values.

In [ ]:
data.isna().sum()

Check for duplicates: The customer might be duplicated given one can make more than 1 transaction.

In [ ]:
data.duplicated(subset = 'TransactionID').sum()

There are no duplicates in the `TransactionID` column.

Drop those subsets with null values for these will not be counted when we group them by Gender, Age, Location, and Account Balance.

In [ ]:
data = data.dropna()

data.isna().sum()

Drop unused field/s.

In [ ]:
data.drop('TransactionTime', axis = 1, inplace = True)

In [ ]:
data.info()

Next, we check `CustAccountBalance` and `TransactionAmount (INR)` for outliers.

In [ ]:
data[['CustAccountBalance', 'TransactionAmount (INR)']].describe().T

Using the describe function, we see some descriptive statistics for our variables. Notice that there are also large max `CustAccountBalance` and `TransactionAmount (INR)` compared to their 75th percentiles.

Check the maximum `CustAccountBalance`.

In [ ]:
data[data['CustAccountBalance'] == data ['CustAccountBalance'].max()]

Check the maximum `TransactionAmount (INR)`.

In [ ]:
data[data['TransactionAmount (INR)'] == data['TransactionAmount (INR)'].max()]

Transform `TransactionDate` and `CustomerDOB` to date time format using `pd.to_datetime()` function.

In [ ]:
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['CustomerDOB'] = pd.to_datetime(data['CustomerDOB'])

Check for the distribution of categorical fields.

In [ ]:
data[['CustomerID', 'CustGender', 'CustLocation']].describe()

Check for the distribution of numerical fields.

In [ ]:
data[['CustAccountBalance', 'TransactionAmount (INR)']].describe()

**Calculate Customer Age**

Here will use the year in the data (2016) as base to get the customer's age

In [ ]:
data['Age'] = data['TransactionDate'].dt.year - data['CustomerDOB'].dt.year

Check for the distribution of `Age`.

In [ ]:
data['Age'].describe()

We can see that there are inconsistencies in the `Age` column. There is a min of -56 and max of 216.

We can transform all the ages below 12 and above 100 percentile into median age.

In [ ]:
data.loc[(data['Age'] < 12) | (data['Age'] >= 100),'Age'] = data['Age'].median()

Check the contents of `CustGender` column.

In [ ]:
data['CustGender'].value_counts()

We can see that the `CustGender` column has one with T value. We can remove this by using the `.replace()` function.

Transform the values of `CustGender` from 'M' to 'Male', 'F' to 'Female' and 'T' to 'Male'.

In [ ]:
data['CustGender'] = data['CustGender'].replace({'M':'Male', 'F':'Female', 'T':'Male'})

In [ ]:
data.sample(10)

## 2. Exploratory Analysis

We can extend our previous exploratory analysis by looking at the following:
1. Distribution of Transactions per Customer (Frequency)
2. Distribution of Revenue per Customer (Monetary)
3. Total Monthly Revenue

Since we are now interested in the monetary aspect of the data, we add the related metrics in our analysis.

In [ ]:
data.head()

Determine the total count of customers.

In [ ]:
data['CustomerID'].nunique()

There are total of 879,385 number of unique customers.

### 2.1 Gender

In [ ]:
data_customers = data.groupby('CustGender')['TransactionID'].nunique()

data_customers

In [ ]:
# Define a function that will add the currency symbol to the x-axis major tick labels.
def currency(x, pos):
    """The two arguments are the value and tick position"""
    if x >= 1e6:
        s = '₹{:1.1f}M'.format(x*1e-6)
    elif x >= 1e3:
        s = '₹{:1.0f}K'.format(x*1e-3)
    else:
        s = '₹{:1.0f}'.format(x)
    return s

fig, axs = plt.subplots(ncols = 2,nrows = 1, figsize = (15, 8))

axs[0].pie(data['CustGender'].value_counts(), autopct= '%1.f%%', labels = ['Male', 'Female'], startangle = 90, colors = ['dodgerblue', 'tomato'])
axs[0].set_title("Customer Gender Frequency", size = 20)

axs[1] = sns.distplot(data[(data['CustGender'] == 'Male') & (data['TransactionAmount (INR)'] < np.percentile(data['TransactionAmount (INR)'], 90))]['TransactionAmount (INR)'],
                     label = True, kde = False, color = 'dodgerblue')
axs[1] = sns.distplot(data[(data['CustGender'] == 'Female') & (data['TransactionAmount (INR)'] < np.percentile(data['TransactionAmount (INR)'],90))]['TransactionAmount (INR)'],
                     label = 'Female', kde = False, color = 'tomato')
axs[1].set_title("Transaction Amount by Customer Gender", size = 20)

import matplotlib.ticker as ticker
axs[1].xaxis.set_major_formatter(currency)
axs[1].yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.show()

### 2.2 Frequency

Select the top 10 cities with most number of transactions.

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(15, 5))

sns.countplot(y='CustLocation', data=data,
              order=data['CustLocation'].value_counts()[:10].index, ax=ax)

ax.set_title("Top 10 Locations with Most Transactions", fontsize='20')
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_xticklabels([])
ax.xaxis.grid(False)

# Remove figure border
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add data labels
for p in ax.patches:
    ax.annotate(format(int(p.get_width()), ','),
                (p.get_width(), p.get_y() + p.get_height() / 2),
                xytext=(5, 0), textcoords='offset points', ha='left', va='center')

plt.show()


As seen from the plot above, Mumbai, New Delhi, Bangalore, Gurgaon, and Delhi are the top 5 cities with the most number of transactions. This could be due to the that the 3 cities have more population, better economic conditions, and a higher salary range.

### 2.3 Age

In [ ]:
bins = [0, 20, 30, 40, 50, 60, 100]
labels = ['0 - 20', '20 - 30', '30 - 40', '40 - 50', '50 - 60', '60+']
data['AgeBin'] = pd.cut(x = data['Age'], bins = bins, labels = labels, right = True)

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize = (10, 5))
sns.countplot(data, y = 'AgeBin', order = labels)

ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_title('Count of Customers by Age')
ax.set_xticklabels([])
ax.xaxis.grid(False)

# Add data labels at the end of the bars
for p in ax.patches:
    width = p.get_width()
    label = format(int(width), ',')  # Convert width to integer before formatting
    x = width + 10000 if width < 700000 else width - 10000  # Adjusted x-coordinate for label position
    ax.annotate(label,
                xy=(x, p.get_y() + p.get_height() / 2),
                xytext=(5, 0),
                textcoords="offset points",
                ha='left' if width < 700000 else 'right',  # Adjusted horizontal alignment
                va='center')

plt.show()

## 3. Customer Segmentation

### 3.1 RFM Segmentation

RFM model is commonly used in marketing to segment customers based on their shopping behaviors, then treat each segment with targeted strategies. The three metrics used in the segmentation are:

1. **Recency:** How many days since the customer's last transaction date? The lower the value, the more loyal the customers are to the company.
2. **Frequency:** How many times did the customer make transactions during the period? The higher the value, the more active the customers are to our products and services.
3. **Monetary:** The total amount of transactions or money spent by the customer during the period. This is the most important metric in the model. The higher the value, the more monetary value the customer could bring to the company.

**Steps of RFM:**

**Step 1:** Calculate the raw value of each metric.

**Step 2:** Assign a mark to each raw value based on their distributions.

**Step 3:** Based on the average mark of each metric, decide the class of each customer record (0 or 1, 1 means qualified, 0 means unqualified).

**Step 4:** Segment the customers based on their assigned class (0 or 1).

**NOTE:** The time range selected should be decided by the business team based in a real setting, here just using the whole timeframe in the data. Here we do not take the balance into consideration, given the real transaction would create more value for the business.

#### Step 1: Calculate the raw value of each metrics:

In [ ]:
data_RFM = data.copy()
data_RFM = data_RFM.groupby(['CustomerID']).agg(Last_Trans_Date = ('TransactionDate', 'max'),
                                                M = ('TransactionAmount (INR)', 'sum'),
                                                F = ('TransactionID', 'count')).reset_index()

data_RFM.head()

##### Recency Calculation

Use the last transaction date in the data as base, then calculate the difference between each customer's last transaction date to the base.

In [ ]:
data_RFM['Latest_Trans_Date'] = data_RFM['Last_Trans_Date'].max()
data_RFM['R'] = data_RFM['Latest_Trans_Date'] - data_RFM['Last_Trans_Date']
data_RFM['R'] = (data_RFM['R'] / np.timedelta64(1, 'D')).astype(int)

data_RFM = data_RFM[['CustomerID', 'R', 'F', 'M']]
data_RFM.head()

#### Step 2: Assign mark to each raw value based on their distributions

The marking rule or method is arbitary and should be decided by both business and domain experts. 

The scoring is based on the five bins of the distribution of each metric.

In [ ]:
data_RFM['R_Score'] = np.select(
    [data_RFM['R'] <= 50, data_RFM['R'] <= 100, data_RFM['R'] <= 120, data_RFM['R'] <= 150],
    [5, 4, 3, 2],
    default = 1
)

data_RFM['F_Score'] = np.select(
    [data_RFM['F'] == 5, data_RFM['F'] == 4, data_RFM['F'] == 3, data_RFM['F'] == 2],
    [5, 4, 3, 2],
    default = 1
)

data_RFM['M_Score'] = np.select(
    [data_RFM['M'] > 2000, data_RFM['M'] > 1000, data_RFM['M'] > 500, data_RFM['M'] > 300],
    [5, 4, 3, 2],
    default = 1
)

data_RFM = data_RFM[['CustomerID', 'R', 'R_Score', 'F', 'F_Score', 'M', 'M_Score']]
data_RFM.sample(10)

#### The marking strategy is shown below:

![rfm](https://drive.google.com/uc?export=view&id=1NCYfXzqg2NGKynvk5hoJDkXqUHN-hgPm)

In [ ]:
data_RFM.describe().T

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots()

sns.histplot(data_RFM['R_Score'], ax = ax, label = 'R Score', color='blue', alpha = 0.5)
sns.histplot(data_RFM['F_Score'], ax = ax, label = 'F Score', color='orange', alpha = 0.5)
sns.histplot(data_RFM['M_Score'], ax = ax, label = 'M Score', color='green', alpha = 0.5)

ax.legend()
plt.xticks(range(1, 6))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.set_ylabel(None)
ax.set_xlabel(None)
ax.set_title('RFM Scores Distribution')
plt.show()

From the statistics above, most of the scores of the Monetary, Recency and Frequency metrics are gathered at 1, this shows most customers are ordinary and small group of wealthy customers are highly affecting the whole distribution.

Based on the average of each metric score, decide the class of each customer record.

We will use the Mean as the decision base and assign all records higher or equal to the Mean to Class 1 and records lower than the Mean to Class 0.

The base could be Mean as well, depending on the score distribution and business requirement.

In this code, we calculate the means of the `'R_Score'`, `'F_Score'`, and `'M_Score'` columns using the `.mean()` method. Then, we use vectorized comparisons `(>=)` to compare each element in the respective columns with the corresponding mean. The resulting boolean values are converted to integers using `.astype(int)` and then to strings using `.astype(str)`. Finally, the desired columns are selected and stored in the `data_RFM` DataFrame.

By leveraging vectorized operations, you can perform the calculations efficiently without the need for lambda functions or explicit loops, leading to improved performance.

In [ ]:
mean_R_score = data_RFM['R_Score'].mean()
mean_F_score = data_RFM['F_Score'].mean()
mean_M_score = data_RFM['M_Score'].mean()

data_RFM['R'] = (data_RFM['R_Score'] >= mean_R_score).astype(int).astype(str)
data_RFM['F'] = (data_RFM['F_Score'] >= mean_F_score).astype(int).astype(str)
data_RFM['M'] = (data_RFM['M_Score'] >= mean_M_score).astype(int).astype(str)

data_RFM = data_RFM[['CustomerID', 'R', 'F', 'M']]
data_RFM.head()

#### Segment the customers based on their classes

Below is the criteria used for customer segmentation.

**Note:** Criteria is arbitrary and will depend on the business' domain experts and/or managament.

![segmentation_criteria](https://drive.google.com/uc?export=view&id=1IlT1UN-pA2dw38Tk6PAdHFTyfCHj4Pb1)

In [ ]:
data_RFM['RFM'] = data_RFM['R'] + data_RFM['F'] + data_RFM['M']

conditions = [
    (data_RFM['RFM'] == '111'),
    (data_RFM['RFM'] == '101'),
    (data_RFM['RFM'] == '110'),
    (data_RFM['RFM'] == '011'),
    (data_RFM['RFM'] == '001'),
    (data_RFM['RFM'] == '100'),
    (data_RFM['RFM'] == '010'),
    (data_RFM['RFM'] == '000')
]

values = [
    'Most Valuable Customer',
    'Important Developing Customer',
    'Ordinary Customer',
    'Important Maintaining Customer',
    'Important Recalling Customer',
    'Ordinary Developing Customer',
    'Ordinary Maintaining Customer',
    'Ordinary Recalling Customer'
]

data_RFM['Class'] = np.select(conditions, values, default = '')

In [ ]:
data_RFM.sample(5)

#### Visualize the classes distribution: 

In [ ]:
plt.figure(figsize=(12, 5))

ax = sns.countplot(data=data_RFM, y='Class', order=data_RFM['Class'].value_counts().index)

abs_values = data_RFM['Class'].value_counts(ascending=False)
rel_values = data_RFM['Class'].value_counts(ascending=False, normalize=True).values * 100
lbls = [f'{p[0]:,} ({p[1]:.0f}%)' for p in zip(abs_values, rel_values)]

for i, p in enumerate(ax.patches):
    width = p.get_width()
    ax.text(width + 5, p.get_y() + p.get_height() / 2, lbls[i], va='center', size=12)

plt.title('Class Distributions')
plt.ylabel(None)
plt.xlabel(None)
plt.yticks(size=10)

ax.set(xticks=[])

plt.show()


# 2. Pareto Analysis

Pareto analysis, also known as the 80/20 rule or the Pareto principle, is a technique used in decision-making and problem-solving to prioritize and focus efforts on the most significant factors or issues that have the greatest impact. It is named after Italian economist Vilfredo Pareto, who observed that approximately 80% of the wealth in Italy was owned by 20% of the population.

Let us check if the valuable customers have made the most contribution to the transaction volume.

Group by RFM class and sum up the transaction volume of each class.

In [ ]:
from matplotlib.ticker import PercentFormatter

pareto_df = data.groupby(['CustomerID']).agg(Transaction_Amt = ('TransactionAmount (INR)', 'sum')).reset_index()

pareto_df.head()

Sort values in descending order

In [ ]:
pareto_df = pareto_df.merge(data_RFM[['CustomerID', 'Class']], on = 'CustomerID')
pareto_df = pareto_df[['Class', 'Transaction_Amt']].groupby('Class').sum()
pareto_df = pareto_df.sort_values(by = 'Transaction_Amt', ascending = False)

Add cumulative percentage column

In [ ]:
pareto_df['Trans_Percent'] = round(pareto_df["Transaction_Amt"] / pareto_df["Transaction_Amt"].sum() * 100, 2)
pareto_df["Cum_Percent"] = round(pareto_df["Transaction_Amt"].cumsum() / pareto_df["Transaction_Amt"].sum() * 100, 2)
print(pareto_df)

#### Create the Pareto Chart:


The Pareto Chart below visualizes how the transaction volume is accumulated across the RFM classes.

From the chart, we can conclude that the `Important Developing Customer`, `Most Valuable Customer`, and `Important Recalling Customer` classes have contributed to over 90% total transaction volume. These customers account for 50% of the bank's overall customer populatiuon. Even if the number is not 20%, it still shows the important of customer segmentation and targeted service. If we treat the volume as the core KPI, we should focus most our resources providing better products and services to these classes and

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

# Bar plot of transaction volume
ax.bar(pareto_df.index, pareto_df["Transaction_Amt"], color = 'slateblue')
ax.set_title("Pareto Chart of RFM Customer Classes", size = 25, y = 1.05)
ax.set_xlabel("Transaction Volume")
ax.set_ylabel("Frequency")
ax.grid(None)

# Line plot of accumulated volume percentage
ax2 = ax.twinx()
ax2.plot(pareto_df.index, pareto_df["Cum_Percent"], color = "limegreen", marker = "o", ms = 8)
ax2.axhline(80, color = "orange", linestyle = "dashed")
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylabel("Cumulative Percentage")

# Wrap x-axis tick labels
import textwrap
wrapped_labels = [textwrap.fill(label, 15) for label in pareto_df.index]
ax.set_xticklabels(wrapped_labels, size = 12)

# Enlarge and reposition the arrow
annotation_text = "{:,} (20%)".format(int(0.2 * data_RFM.shape[0]))
ax2.annotate("", xy = (0.4, 0.8), xytext = (0.43, 0.8),
            xycoords = 'axes fraction', textcoords = 'axes fraction',
            arrowprops=dict(arrowstyle="-[,widthB = 4.0, lengthB = 0.8", lw = 2.5, color = 'black'),
            annotation_clip=False)


ax2.annotate(annotation_text, xy = (0.53, 0.79),
            xycoords = 'axes fraction', fontsize = 15, ha = 'center')

plt.show()